In [1]:
# !pip install translators gTTS pydub
# !pip install playsound3
# !pip install -U deep_translator
!pip install pyttsx3

In [ ]:
import pyttsx3
import time

taxa_desejada = 280
texto = "Meu nome é Guilherme"

engine = pyttsx3.init()

# A linha abaixo pode causar erro se não houver vozes ou se o engine não estiver totalmente pronto.
# Para este problema específico do 'aplay', ela não é a causa direta, mas é bom saber.
# Se você ainda tiver problemas APÓS instalar o alsa-utils, pode tentar comentar a linha das vozes
# temporariamente para isolar o problema, embora idealmente ela deva funcionar.
# voices = engine.getProperty('voices') # Removido temporariamente para simplificar o teste do aplay

engine.setProperty('rate', taxa_desejada) 

print(f"Tentando falar: '{texto}' com taxa: {taxa_desejada}")

start_time = time.time()
engine.say(texto)
engine.runAndWait() 
end_time = time.time()

duration = end_time - start_time
print(f"Duração aproximada da fala: {duration:.2f} segundos")

Tentando falar: 'Meu nome é Guilherme' com taxa: 280
Duração aproximada da fala: 0.01 segundos


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM default
aplay: main:834: audio open error: No such file or directory


In [2]:
import pyttsx3
import time

def falar_com_taxa_ajustada(texto, taxa_desejada, idioma='pt-br'):
    engine = pyttsx3.init()
    
    # Lista de vozes disponíveis e configuração para português (se disponível)
    voices = engine.getProperty('voices')
    for voice in voices:
        if idioma in voice.languages or any(lang.decode('utf-8', 'ignore').split('_')[0] == idioma.split('-')[0] for lang in voice.languages):
            engine.setProperty('voice', voice.id)
            break
            
    engine.setProperty('rate', taxa_desejada) 
    
    print(f"Tentando falar '{texto}' com taxa: {taxa_desejada}")
    
    # Medir o tempo (aproximado, pois engine.runAndWait() é bloqueante)
    # Para uma medição mais precisa, você salvaria em arquivo e mediria a duração do arquivo.
    start_time = time.time()
    engine.say(texto)
    engine.runAndWait()
    end_time = time.time()
    
    duration = end_time - start_time
    print(f"Duração aproximada da fala: {duration:.2f} segundos")
    return duration

frase = "Meu nome é Guilherme"

# --- Processo de Tentativa e Erro ---
# A taxa padrão é geralmente em torno de 200.
# Valores maiores são mais rápidos, menores são mais lentos.
# Você precisará ajustar 'taxa_alvo' experimentalmente.

# Exemplo:
# taxa_experimental = 200 # Comece com um valor e ajuste
# duracao_obtida = falar_com_taxa_ajustada(frase, taxa_experimental)
# print(f"Para taxa {taxa_experimental}, a duração foi {duracao_obtida:.2f}s")

# Suponha que após alguns testes, você encontrou uma taxa que se aproxima:
taxa_ideal_aproximada = 280 # Este valor é um CHUTE, você precisará encontrar o seu!
print("\n--- Demonstração com taxa 'ideal' (experimental) ---")
falar_com_taxa_ajustada(frase, taxa_ideal_aproximada)

# Para salvar em arquivo e medir com mais precisão:
# engine = pyttsx3.init()
# engine.setProperty('rate', taxa_ideal_aproximada)
# # ... (configurar voz)
# engine.save_to_file(frase, 'fala_teste.wav') # ou .mp3 dependendo do engine
# engine.runAndWait()
# # Depois, use uma biblioteca como pydub ou librosa para obter a duração exata de 'fala_teste.wav'


--- Demonstração com taxa 'ideal' (experimental) ---


RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!

In [ ]:
import re
import os
import time
from datetime import datetime, timedelta
# import translators as ts
from gtts import gTTS
from pydub import AudioSegment # <--- Adicionado para duração
from pydub.exceptions import CouldntDecodeError # <--- Para tratar erros do pydub
try:
    from playsound3 import playsound # <--- Adicionado para tocar áudio
except ImportError:
    print("AVISO: Biblioteca 'playsound' não encontrada. A reprodução de áudio será desabilitada.")
    print("Instale com: pip install playsound")
    playsound = None # Define playsound como None se não puder ser importado

from pydub.effects import speedup as pydub_speedup



# --- Configurações ---
# (Mantenha suas configurações existentes)
TRANSCRIPT_FILE_PATH = "data/transcricao_com_marcação_de_tempo_exemple.txt"
OUTPUT_AUDIO_DIR = "audio_dublado_pt"
TARGET_LANGUAGE = 'pt' # Português
SOURCE_LANGUAGE = 'en' # Inglês

# --- Funções Auxiliares ---
# (Mantenha parse_timestamp, get_transcript_content, segment_transcript, translate_segments)
# ... (seu código anterior para essas funções) ...
from deep_translator import GoogleTranslator

translated = GoogleTranslator(source='en', target='pt')


def parse_timestamp(ts_str):
    """Converte uma string de timestamp HH:MM:SS para um objeto timedelta."""
    try:
        h, m, s = map(int, ts_str.split(':'))
        return timedelta(hours=h, minutes=m, seconds=s)
    except ValueError:
        print(f"AVISO: Timestamp inválido encontrado e ignorado: {ts_str}")
        return None

def get_transcript_content(file_path):
    """Lê o conteúdo do arquivo de transcrição."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"ERRO: Arquivo de transcrição não encontrado em '{file_path}'.")
        return None



def segment_transcript(transcript_content):
    """Segmenta a transcrição."""
    if not transcript_content:
        return []
    pattern = re.compile(r"(\d{2}:\d{2}:\d{2})(.*?(?=\d{2}:\d{2}:\d{2}|$))", re.DOTALL)
    matches = pattern.findall(transcript_content)
    segments = []
    for i, (timestamp_str, text_en_raw) in enumerate(matches):
        start_time_obj = parse_timestamp(timestamp_str)
        if start_time_obj is None: continue
        text_en = text_en_raw.strip()
        if not text_en: continue
        end_time_obj = None
        if i + 1 < len(matches):
            next_timestamp_str, _ = matches[i+1]
            end_time_obj = parse_timestamp(next_timestamp_str)
        segments.append({
            "id": i, "start_time_str": timestamp_str, "start_time_td": start_time_obj,
            "end_time_td": end_time_obj, "text_en": text_en, "text_pt": None,
            "audio_file_path": None, "audio_duration_sec": None # <--- Adicionado audio_duration_sec
        })
    return segments



def translate_segments(segments):
    """Traduz o texto de cada segmento."""
    print("\n--- Iniciando Tradução ---")
    for segment in segments:
        if segment["text_en"]:
            try:
                segment["text_pt"] = translated.translate(segment["text_en"])  
                # ts.translate_text(
                #     segment["text_en"], translator='google',
                #     from_language=SOURCE_LANGUAGE, to_language=TARGET_LANGUAGE
                # )
                print(f"  Segmento {segment['id']} ({segment['start_time_str']}): Traduzido para PT.")
            except Exception as e:
                print(f"  ERRO na tradução do segmento {segment['id']}: {e}")
                segment["text_pt"] = f"[Erro na tradução: {segment['text_en']}]"
        else:
            segment["text_pt"] = ""
    print("--- Tradução Concluída ---")
    return segments

def manage_speed_adjusted_audio(sound, original_duration_sec, target_slot_duration_sec):
    target_slot_duration_sec = target_slot_duration_sec + 1.5

    playback_speed_factor = original_duration_sec / target_slot_duration_sec

    # Determina se o ajuste é necessário e está dentro dos limites aceitáveis
    adjusted_sound = pydub_speedup(sound, playback_speed=playback_speed_factor, chunk_size=150, crossfade=25)
    
    return adjusted_sound

def generate_tts_for_segments(segments, output_dir):
    """Gera arquivos de áudio e obtém sua duração."""
    print("\n--- Iniciando Geração de Áudio (TTS) ---")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for segment in segments:
        if segment["text_pt"]:
            try:
                tts = gTTS(text=segment["text_pt"], lang=TARGET_LANGUAGE, slow=False)
                safe_timestamp_str = segment['start_time_str'].replace(':', '_')
                audio_filename = f"segment_{segment['id']:03d}_{safe_timestamp_str}.mp3"
                audio_path = os.path.join(output_dir, audio_filename)

                audio_segment_obj = AudioSegment.from_mp3(audio_path)

                target_slot_duration_sec = segment["end_time_td"] - segment["start_time_td"]

                # audio_segment_obj = manage_speed_adjusted_audio(sound=audio_segment_obj, original_duration_sec=len(audio_segment_obj) / 1000.0, 
                #                             target_slot_duration_sec=target_slot_duration_sec)

                tts.save(audio_path)
                segment["audio_file_path"] = audio_path
                print(f"  Segmento {segment['id']}: Áudio salvo em '{audio_path}'")

                # Obter duração do áudio usando pydub
                try:
                    


                    segment["audio_duration_sec"] = len(audio_segment_obj) / 1000.0 # pydub retorna em milissegundos
                    print(f"    Duração do áudio: {segment['audio_duration_sec']:.2f}s")
                except CouldntDecodeError:
                    print(f"    AVISO: pydub não conseguiu decodificar '{audio_path}'. Verifique se o FFmpeg está instalado e no PATH.")
                    segment["audio_duration_sec"] = None # Define como None se não puder obter
                except Exception as e_pydub:
                    print(f"    AVISO: Não foi possível obter a duração do áudio para {audio_filename} (pydub): {e_pydub}")
                    segment["audio_duration_sec"] = None

            except Exception as e:
                print(f"  ERRO no TTS para segmento {segment['id']}: {e}")
                segment["audio_file_path"] = None
                segment["audio_duration_sec"] = None
        else:
            print(f"  Segmento {segment['id']}: Sem texto em português para gerar áudio.")
            segment["audio_file_path"] = None
            segment["audio_duration_sec"] = None
    print("--- Geração de Áudio Concluída ---")
    return segments

def simulate_synchronized_playback(segments):
    """Simula a reprodução dos áudios sincronizados com os timestamps."""
    print("\n--- Iniciando Simulação de Playback Sincronizado com Áudio Real ---")
    if not segments:
        print("Nenhum segmento para simular.")
        return

    if playsound is None:
        print("Playback de áudio desabilitado pois a biblioteca 'playsound' não foi carregada.")
        # Fallback para simulação sem áudio real se playsound não estiver disponível
        # (ou pode optar por parar aqui)

    print("Instruções: Inicie seu vídeo do YouTube em 00:00:00.")
    print("O script tocará cada áudio traduzido no momento indicado.")
    print("Pressione Enter para começar a simulação...")
    input()

    start_real_time = datetime.now()

    for i, segment in enumerate(segments):
        target_play_time_td = segment["start_time_td"]
        delay_from_sim_start_sec = (target_play_time_td - timedelta(0)).total_seconds()
        elapsed_real_time_sec = (datetime.now() - start_real_time).total_seconds()
        wait_duration_sec = max(0, delay_from_sim_start_sec - elapsed_real_time_sec)

        if wait_duration_sec > 0:
            print(f"  Aguardando {wait_duration_sec:.2f}s para o próximo segmento (timestamp: {segment['start_time_str']})...")
            time.sleep(wait_duration_sec)
        
        # Atualiza o tempo de referência real para o momento em que o segmento deveria começar
        # Isso ajuda a corrigir pequenos desvios acumulados pelo `time.sleep` e processamento
        current_processing_time_taken = (datetime.now() - start_real_time).total_seconds() - delay_from_sim_start_sec
        start_real_time += timedelta(seconds=current_processing_time_taken)


        print(f"\n[{segment['start_time_str']} / Tempo real aprox.: {datetime.now().strftime('%H:%M:%S')}]")
        if segment["audio_file_path"] and playsound:
            print(f"  ▶️  TOCANDO ÁUDIO: {segment['audio_file_path']}")
            print(f"     Texto Original: \"{segment['text_en'][:80]}...\"")
            print(f"     Texto Traduzido: \"{segment['text_pt'][:80]}...\"")
            if segment["audio_duration_sec"] is not None:
                print(f"     Duração Estimada do Áudio: {segment['audio_duration_sec']:.2f}s")

            try:
                # playsound é bloqueante por padrão, o script espera o áudio terminar.
                playsound(segment["audio_file_path"])
                print(f"  ⏹️  ÁUDIO TERMINADO: {segment['audio_file_path']}")

            # except PlaysoundException as pse:
            #     print(f"    ERRO ao tocar áudio com playsound: {pse}")
            #     print(f"    Verifique se você tem um backend de áudio compatível instalado (ex: mpg123, GStreamer) ou se o arquivo não está corrompido.")
            except Exception as e_play: # Captura outras exceções potenciais
                print(f"    ERRO desconhecido ao tentar tocar áudio: {e_play}")

        elif not playsound and segment["audio_file_path"]:
             print(f"  (Simulando) ▶️  TOCAR ÁUDIO: {segment['audio_file_path']} (playsound não disponível)")
             if segment["audio_duration_sec"] is not None:
                print(f"     Duração Estimada do Áudio: {segment['audio_duration_sec']:.2f}s")
                time.sleep(segment["audio_duration_sec"]) # Simula espera se playsound não estiver lá
                print(f"  (Simulando) ⏹️  ÁUDIO TERMINADO: {segment['audio_file_path']}")
             else:
                print(f"     (Duração do áudio não pôde ser determinada)")

        elif not segment["audio_file_path"]:
            print(f"  ⚠️  Sem áudio para tocar para o segmento {segment['id']}.")

        # Informação sobre o segmento original no vídeo
        if segment.get("end_time_td"):
            duration_of_original_segment_video = (segment["end_time_td"] - segment["start_time_td"]).total_seconds()
            print(f"     (Segmento original no vídeo dura ~{duration_of_original_segment_video:.2f}s até o próximo timestamp)")
        else: # Último segmento
            if segment["audio_duration_sec"]:
                 print(f"     (Este é o último segmento. Áudio dublado dura ~{segment['audio_duration_sec']:.2f}s)")


    print("\n--- Simulação de Playback Concluída ---")


# --- Execução Principal ---
if __name__ == "__main__":
    print("### INICIANDO PROCESSO DE DUBLAGEM COM ÁUDIO REAL (SIMULADO) ###")
    transcript_data = get_transcript_content(TRANSCRIPT_FILE_PATH)
    if transcript_data:
        parsed_segments = segment_transcript(transcript_data)
        if not parsed_segments:
            print("Nenhum segmento válido encontrado na transcrição.")
        else:
            print(f"\n{len(parsed_segments)} segmentos parseados.")
            # Removido print do primeiro segmento para encurtar a saída padrão
            translated_segments = translate_segments(parsed_segments)
            segments_with_audio = generate_tts_for_segments(translated_segments, OUTPUT_AUDIO_DIR)
            simulate_synchronized_playback(segments_with_audio)
            print("\nProcesso finalizado.")
            print(f"Os arquivos de áudio foram salvos em: '{os.path.abspath(OUTPUT_AUDIO_DIR)}'")
    else:
        print("Não foi possível carregar os dados da transcrição. Encerrando.")

### INICIANDO PROCESSO DE DUBLAGEM COM ÁUDIO REAL (SIMULADO) ###

20 segmentos parseados.

--- Iniciando Tradução ---
  Segmento 1 (00:00:00): Traduzido para PT.
  Segmento 2 (00:00:01): Traduzido para PT.
  Segmento 3 (00:00:02): Traduzido para PT.
  Segmento 4 (00:00:04): Traduzido para PT.
  Segmento 5 (00:00:06): Traduzido para PT.
  Segmento 6 (00:00:08): Traduzido para PT.
  Segmento 7 (00:00:10): Traduzido para PT.
  Segmento 8 (00:00:12): Traduzido para PT.
  Segmento 9 (00:00:14): Traduzido para PT.
  Segmento 10 (00:00:16): Traduzido para PT.
  Segmento 11 (00:00:18): Traduzido para PT.
  Segmento 12 (00:00:20): Traduzido para PT.
  Segmento 13 (00:00:21): Traduzido para PT.
  Segmento 14 (00:00:22): Traduzido para PT.
  Segmento 15 (00:00:24): Traduzido para PT.
  Segmento 16 (00:00:26): Traduzido para PT.
  Segmento 17 (00:00:29): Traduzido para PT.
  Segmento 18 (00:00:30): Traduzido para PT.
  Segmento 19 (00:00:32): Traduzido para PT.
  Segmento 20 (00:00:34): Traduzido 

In [39]:
from gtts import gTTS
from pydub import AudioSegment
from pydub.effects import speedup as pydub_speedup
import os

# Opcional: para tocar o som diretamente no script
try:
    from playsound3 import playsound
except ImportError:
    print("AVISO: Biblioteca 'playsound3' não encontrada. O áudio não será tocado automaticamente.")
    playsound = None

def gerar_audio_com_duracao_especifica(texto: str, idioma: str, duracao_alvo_seg: float, nome_arquivo_saida: str = "audio_ajustado.mp3", gtts_lento: bool = False):
    """
    Gera um arquivo de áudio a partir de um texto com uma duração alvo específica.

    Args:
        texto (str): O texto a ser convertido em fala.
        idioma (str): O código do idioma para o gTTS (ex: 'pt' para português).
        duracao_alvo_seg (float): A duração desejada para o áudio final em segundos.
        nome_arquivo_saida (str): Nome do arquivo MP3 de saída.
        gtts_lento (bool): Usar o modo 'slow' do gTTS para o áudio base.

    Returns:
        tuple: (caminho_do_arquivo_final, duracao_real_final_seg) ou (None, None) em caso de erro.
    """
    arquivo_tts_temporario = "temp_tts_inicial.mp3"

    # 1. Gerar áudio inicial com gTTS
    try:
        print(f"Gerando áudio base com gTTS (lento={gtts_lento})...")
        tts = gTTS(text=texto, lang=idioma, slow=gtts_lento)
        tts.save(arquivo_tts_temporario)
        print(f"Áudio base salvo em: {arquivo_tts_temporario}")
    except Exception as e:
        print(f"ERRO ao gerar áudio base com gTTS: {e}")
        return None, None

    # 2. Carregar com pydub e obter duração original
    try:
        som_original = AudioSegment.from_mp3(arquivo_tts_temporario)
        duracao_original_seg = som_original.duration_seconds
        print(f"Duração original do áudio base: {duracao_original_seg:.3f}s")
    except Exception as e:
        print(f"ERRO ao carregar áudio base com pydub: {e}")
        if os.path.exists(arquivo_tts_temporario):
            os.remove(arquivo_tts_temporario)
        return None, None

    if duracao_original_seg <= 0:
        print("ERRO: Áudio base gerado tem duração inválida (zero ou negativa).")
        if os.path.exists(arquivo_tts_temporario):
            os.remove(arquivo_tts_temporario)
        return None, None
        
    # 3. Calcular fator de velocidade para pydub.effects.speedup
    # playback_speed > 1.0 acelera, < 1.0 desacelera.
    # Se original=10s, alvo=5s => playback_speed = 10/5 = 2.0 (acelerar 2x)
    # Se original=5s, alvo=10s => playback_speed = 5/10 = 0.5 (desacelerar para metade da velocidade)
    fator_de_velocidade = duracao_original_seg / duracao_alvo_seg
    print(f"Fator de velocidade calculado: {fator_de_velocidade:.3f} (para alvo de {duracao_alvo_seg:.2f}s)")

    som_ajustado = None

    # Limites práticos para o fator de velocidade para evitar distorção excessiva
    LIMITE_MIN_FATOR = 0.5  # Não mais lento que metade da velocidade original
    LIMITE_MAX_FATOR = 2.5  # Não mais rápido que 2.5x a velocidade original

    if LIMITE_MIN_FATOR <= fator_de_velocidade <= LIMITE_MAX_FATOR:
        try:
            print(f"Ajustando velocidade do áudio...")
            # Parâmetros chunk_size e crossfade podem ser ajustados para qualidade vs velocidade de processamento
            som_ajustado = pydub_speedup(som_original, playback_speed=fator_de_velocidade, chunk_size=150, crossfade=25)
        except Exception as e:
            print(f"ERRO ao aplicar ajuste de velocidade: {e}.")
            print("O áudio final será o original sem ajuste de velocidade.")
            som_ajustado = som_original # Usa o original como fallback
    else:
        print(f"AVISO: Fator de velocidade ({fator_de_velocidade:.3f}) está fora dos limites aceitáveis ({LIMITE_MIN_FATOR}-{LIMITE_MAX_FATOR}).")
        # Tenta aplicar o ajuste com o fator "clampado" ao limite mais próximo
        fator_clampado = max(LIMITE_MIN_FATOR, min(LIMITE_MAX_FATOR, fator_de_velocidade))
        print(f"Tentando ajuste com fator de velocidade limitado a: {fator_clampado:.3f}")
        try:
            som_ajustado = pydub_speedup(som_original, playback_speed=fator_clampado, chunk_size=150, crossfade=25)
            print("AVISO: Devido aos limites, a duração final pode não ser exatamente a alvo.")
        except Exception as e:
            print(f"ERRO ao aplicar ajuste de velocidade com fator limitado: {e}.")
            print("O áudio final será o original sem ajuste de velocidade.")
            som_ajustado = som_original # Usa o original como fallback

    if som_ajustado is None: # Salvaguarda caso algo dê muito errado
        print("Falha crítica no ajuste, usando som original.")
        som_ajustado = som_original

    # 4. Exportar o áudio ajustado (ou o original, se o ajuste falhou)
    try:
        som_ajustado.export(nome_arquivo_saida, format="mp3")
        duracao_final_seg = som_ajustado.duration_seconds
        print(f"Áudio final salvo em: '{nome_arquivo_saida}', Duração final: {duracao_final_seg:.3f}s")
    except Exception as e:
        print(f"ERRO ao exportar áudio final: {e}")
        if os.path.exists(arquivo_tts_temporario):
            os.remove(arquivo_tts_temporario)
        return None, None
    
    # 5. Limpar arquivo temporário
    if os.path.exists(arquivo_tts_temporario):
        os.remove(arquivo_tts_temporario)

    return nome_arquivo_saida, duracao_final_seg

# --- Exemplo de Uso ---
if __name__ == "__main__":
    texto_falado = "Isso é um teste"
    idioma_falado = 'pt'
    duracao_desejada_seg = 3.0
    # Usar gtts_lento=True conforme seu exemplo, para o áudio base ser mais longo.
    # Se a duracao_desejada_seg for curta, o fator_de_velocidade será alto (acelerar bastante).
    usar_gtts_lento = True 
    nome_arquivo = "meu_audio_de_3_segundos.mp3"

    caminho_final, duracao_real = gerar_audio_com_duracao_especifica(
        texto=texto_falado,
        idioma=idioma_falado,
        duracao_alvo_seg=duracao_desejada_seg,
        nome_arquivo_saida=nome_arquivo,
        gtts_lento=usar_gtts_lento
    )

    if caminho_final:
        print(f"\nProcesso concluído!")
        print(f"Arquivo gerado: {caminho_final}")
        print(f"Duração real obtida: {duracao_real:.3f} segundos (Alvo era: {duracao_desejada_seg:.1f}s)")

        if playsound and os.path.exists(caminho_final):
            print("\nTocando o áudio gerado...")
            try:
                playsound(caminho_final)
            except Exception as e:
                print(f"Erro ao tentar tocar o áudio com playsound3: {e}")
    else:
        print("\nFalha ao gerar o áudio com duração específica.")

Gerando áudio base com gTTS (lento=True)...
Áudio base salvo em: temp_tts_inicial.mp3
Duração original do áudio base: 1.992s
Fator de velocidade calculado: 0.664 (para alvo de 3.00s)
Ajustando velocidade do áudio...
Áudio final salvo em: 'meu_audio_de_3_segundos.mp3', Duração final: 1.109s

Processo concluído!
Arquivo gerado: meu_audio_de_3_segundos.mp3
Duração real obtida: 1.109 segundos (Alvo era: 3.0s)

Tocando o áudio gerado...



In [ ]:
def manage_speed_adjusted_audio(sound, original_duration_sec, target_slot_duration_sec):
    target_slot_duration_sec = target_slot_duration_sec + 1.5

    playback_speed_factor = original_duration_sec / target_slot_duration_sec

    # Determina se o ajuste é necessário e está dentro dos limites aceitáveis
    adjusted_sound = pydub_speedup(sound, playback_speed=playback_speed_factor, chunk_size=150, crossfade=25)
    
    return adjusted_sound


In [38]:
audio_path = "/home/guilherme/pyprojects/from_zero_to_hero/notebooks/audio_dublado_pt/segment_001_00_00_00.mp3"

tts.save(audio_path)
sound = AudioSegment.from_mp3(audio_path)

# original_duration_sec = sound.duration_seconds
sound = manage_speed_adjusted_audio(sound, original_duration_sec=sound.duration_seconds,
                            target_slot_duration_sec=2)
sound

In [19]:
adjusted_sound

In [24]:
parsed_segments[0]["end_time_td"] - parsed_segments[0]["start_time_td"]

datetime.timedelta(seconds=1)

In [16]:
get_transcript_content(TRANSCRIPT_FILE_PATH)

NameError: name 'get_transcript_content' is not defined